In [1]:
from dotenv import load_dotenv

env_path = "../OllamaSetup/.env"  
load_dotenv(env_path)

True

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


In [3]:
llm= ChatOllama(model='llama3.2:3b',base_url='http://localhost:11434')
llm.invoke("Hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T09:33:12.554353Z', 'done': True, 'done_reason': 'stop', 'total_duration': 335944959, 'load_duration': 26337750, 'prompt_eval_count': 26, 'prompt_eval_duration': 121538250, 'eval_count': 8, 'eval_duration': 187591625, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-4fec395a-c074-4686-ad79-173d3eced3e2-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

## Tool Creation

In [13]:
from langchain_core.tools import tool

@tool
def add(a,b):
    """ 
    Add two integer number together
    Args:
    a: First Integer
    b:Second Integer
    """
    return a+b

In [86]:
@tool
def multiply(a:int,b:int)->int:
    """ 
    Multiply two integer number together
    Args:
    a: First Integer
    b:Second Integer
    """
    return int(a)*int(b)

In [15]:
add

StructuredTool(name='add', description='Add two integer number together\nArgs:\na: First Integer\nb:Second Integer', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x1169ba840>)

In [16]:
add.name,add.description, add.args, add.args_schema.schema()

/var/folders/v_/q_0xfydn1ysbg3dg6_yj25th0000gn/T/ipykernel_3949/2577403487.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  add.name,add.description, add.args, add.args_schema.schema()


('add',
 'Add two integer number together\nArgs:\na: First Integer\nb:Second Integer',
 {'a': {'title': 'A'}, 'b': {'title': 'B'}},
 {'description': 'Add two integer number together\nArgs:\na: First Integer\nb:Second Integer',
  'properties': {'a': {'title': 'A'}, 'b': {'title': 'B'}},
  'required': ['a', 'b'],
  'title': 'add',
  'type': 'object'})

In [19]:
add.invoke({'a':1,'b':2})

3

In [20]:
tools=[add,multiply]
llm_with_tools =llm.bind_tools(tools)

In [21]:
question =" What is one plus two"
llm_with_tools.invoke(question)

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T09:47:22.955326Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2072401334, 'load_duration': 730826875, 'prompt_eval_count': 235, 'prompt_eval_duration': 770811750, 'eval_count': 22, 'eval_duration': 569074208, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-78b381df-c9a0-43b5-91b0-05ba810b7f7f-0', tool_calls=[{'name': 'add', 'args': {'a': '1', 'b': '2'}, 'id': '204bc62b-c26b-4d1e-8710-b4c8cc080eda', 'type': 'tool_call'}], usage_metadata={'input_tokens': 235, 'output_tokens': 22, 'total_tokens': 257})

In [ ]:
question =" What is one plus two"
llm_with_tools.invoke(question).tool_calls

[{'name': 'add',
  'args': {'a': 1, 'b': 2},
  'id': '047741d0-330a-4b1a-87f5-04abf3183957',
  'type': 'tool_call'}]

In [24]:
question =" What is one multiply two ,also 23 plus seven"
llm_with_tools.invoke(question).tool_calls

[{'name': 'multiply',
  'args': {'a': '2', 'b': '1'},
  'id': '5b47887d-4da7-4438-a3d7-92d7abdf23ee',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': '23', 'b': '7'},
  'id': '1127f6b8-b392-40ad-a799-721660e81211',
  'type': 'tool_call'}]

## Built In Tools

## Duckduckgo

In [ ]:
#!pip install -qU duckduckgo-search 

In [26]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

'The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of ... Barack Obama, the 44th President of the United States, broke barriers as the first African-American president and implemented significant healthcare reforms during his tenure. ... Secret Service Code Names of 11 U.S. Presidents. ... President Barack Obama\'s First Words to America" (2009) "The Audacity of Hope: Thoughts on Reclaiming the ... Here is a list of the presidents and vice presidents of the United States along with their parties and dates in office. ... Chester A Arthur: Twenty-First President of the United States. 10 Interesting Facts About James Buchanan. Martin Van Buren - Eighth

## Tavily

In [ ]:
#!pip install -qU langchain-tavily
#!pip install -qU langgraph

In [35]:
from langchain_community.tools import TavilySearchResults


# Initialize Tavily Search Tool
search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
)



In [36]:
question="What is today stock market news"
search.invoke(question)

[{'title': 'Latest Stock Market News - Yahoo Finance',
  'url': 'https://finance.yahoo.com/topic/stock-market-news/',
  'content': "### It's 'economic blackout' day as campaign encourages Americans not to spend for 24 hours A retail boycott is hitting major US businesses Friday as an online campaign calls for Americans to spend nothing at places such as Walmart, Target, Amazon and McDonald’s.\n\n### Stock market today: Dow, S&P 500, Nasdaq rally to cap volatile week, losing month Investors are waiting for the release of the Federal Reserve's preferred inflation gauge as they eye Trump’s latest trade threats. [...] ### Why Tesla's stock has given up nearly all of its Trump election gains Tesla stock has nearly given up all its gains since President Trump’s election win, as Elon Musk’s forays with politics have investors wondering where the CEO’s attention lies.\n\n### Stock market today: Nasdaq, S&P 500 sell-off as Nvidia sinks 8%, Trump reiterates tariff plans An upbeat tone is seeping

## Wikipedia search

In [ ]:
#!pip install --upgrade --quiet  wikipedia

In [38]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [39]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [ ]:
question="What is the capital of France?"
wikipedia.invoke(question)

"Page: Closed-ended question\nSummary: A closed-ended question is any question for which a researcher provides research participants with options from which to choose a response. Closed-ended questions are sometimes phrased as a statement that requires a response.\nA closed-ended question contrasts with an open-ended question, which cannot easily be answered with specific information.\n\n\n\nPage: France\nSummary: France, officially the French Republic, is a country located primarily in Western Europe. Its overseas regions and territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Indies, and many islands in Oceania and the Indian Ocean, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the northeast, Switzerland to the east, Italy and Monaco to the southeast, Andorra and Spain to the south, and a maritime 

In [42]:
question="What is LLM?"
print(wikipedia.invoke(question))

Page: Large language model
Summary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.
The largest and most capable LLMs are generative pretrained transformers (GPTs). Modern models can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.

Page: Vibe coding
Summary: Vibe coding (also vibecoding) is an AI-dependent programming technique where a person describes a problem in a few sentences as a prompt to a  large language model (LLM) tuned for coding. The LLM generates software, shifting the programmer's role from manual coding to guiding, testing, and refining the AI

## PubMed

In [ ]:
#!pip install xmltodict

In [44]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

In [46]:
search = PubmedQueryRun()
print(search.invoke("What causes lung cancer"))

Published: 2025-03-31
Title: Statistical analysis of spatial patterns in tumor microenvironment images.
Copyright Information: © 2025. The Author(s).
Summary::
Advances in tissue labeling, imaging, and automated cell identification now enable the visualization of immune cell types in human tumors. However, a framework for analyzing spatial patterns within the tumor microenvironment (TME) is still lacking. To address this, we develop Spatiopath, a null-hypothesis framework that distinguishes statistically significant immune cell associations from random distributions. Using embedding functions to map cell contours and tumor regions, Spatiopath extends Ripley's K function to analyze both cell-cell and cell-tumor interactions. We validate the method with synthetic simulations and apply it to multi-color images of lung tumor sections, revealing significant spatial patterns such as mast cells accumulating near T cells and the tumor epithelium. These patterns highlight differences in spatial

## tool Calling with LLm

In [ ]:
@tool
def Wikipedia_search(query):
    """ 
    Search wikipedia for general information

    Args: The search query   
    """

    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response

In [48]:
@tool
def pubmed_search(query):
    """ 
    Search pubmed for medical and life sciences queries

    Args: The search query   
    """

    search = PubmedQueryRun()
    response = search.invoke(query)
    return response

In [49]:
@tool
def tavily_search(query):
    """ 
    Search the web for real time and latest information.
    for example news,weather upadate ,stock news etc

    Args: The search query   
    """

    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )

    response = search.invoke(query)
    return response

In [50]:
tools= [Wikipedia_search,pubmed_search,tavily_search]

In [51]:
list_of_tools= {tool.name : tool for tool in tools}

In [53]:
list_of_tools

{'Wikipedia_search': StructuredTool(name='Wikipedia_search', description='Search wikipedia for a query\n\nArgs: The search query', args_schema=<class 'langchain_core.utils.pydantic.Wikipedia_search'>, func=<function Wikipedia_search at 0x12a91f920>),
 'pubmed_search': StructuredTool(name='pubmed_search', description='Search pubmed for medical and life sciences queries\n\nArgs: The search query', args_schema=<class 'langchain_core.utils.pydantic.pubmed_search'>, func=<function pubmed_search at 0x129f87ce0>),
 'tavily_search': StructuredTool(name='tavily_search', description='Search the web for real time and latest information.\nfor example news,weather upadate ,stock news etc\n\nArgs: The search query', args_schema=<class 'langchain_core.utils.pydantic.tavily_search'>, func=<function tavily_search at 0x12ab1dee0>)}

In [55]:
llm_with_tools=llm.bind_tools(tools)

In [ ]:
query= " What is the latest news ? "
response= llm_with_tools.invoke(query)
print(response)
print(response.tool_calls)


content='' additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T11:34:44.812719Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2956099292, 'load_duration': 734399917, 'prompt_eval_count': 285, 'prompt_eval_duration': 1701547709, 'eval_count': 20, 'eval_duration': 518640458, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-a7ed0cfd-7980-4783-adf3-2365969e0f2d-0' tool_calls=[{'name': 'tavily_search', 'args': {'query': 'latest news'}, 'id': 'b3a9e965-8641-47da-8a2a-84707e024af6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 285, 'output_tokens': 20, 'total_tokens': 305}


In [60]:
query= " What is LLM ? "
response= llm_with_tools.invoke(query)
print(response)
print(response.tool_calls)

content='' additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T11:36:28.688301Z', 'done': True, 'done_reason': 'stop', 'total_duration': 732968666, 'load_duration': 27103708, 'prompt_eval_count': 284, 'prompt_eval_duration': 183014542, 'eval_count': 20, 'eval_duration': 522513208, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-dc723e30-082b-4859-8d4a-506a47f43356-0' tool_calls=[{'name': 'tavily_search', 'args': {'query': 'LLM'}, 'id': '0d4fe5a6-fa6c-4a8a-97b1-62f41c03b9fb', 'type': 'tool_call'}] usage_metadata={'input_tokens': 284, 'output_tokens': 20, 'total_tokens': 304}
[{'name': 'tavily_search', 'args': {'query': 'LLM'}, 'id': '0d4fe5a6-fa6c-4a8a-97b1-62f41c03b9fb', 'type': 'tool_call'}]


In [59]:
query= " What is the today's stock market news ?"
response= llm_with_tools.invoke(query)
print(response)
print(response.tool_calls)

content='' additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T11:36:00.534575Z', 'done': True, 'done_reason': 'stop', 'total_duration': 729584666, 'load_duration': 61788791, 'prompt_eval_count': 287, 'prompt_eval_duration': 170207000, 'eval_count': 19, 'eval_duration': 497147458, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-4dae2738-700c-410f-85c2-0b8eb617c57a-0' tool_calls=[{'name': 'tavily_search', 'args': {'query': 'today'}, 'id': 'c2407249-04ce-4cd1-a495-079874acf8e5', 'type': 'tool_call'}] usage_metadata={'input_tokens': 287, 'output_tokens': 19, 'total_tokens': 306}
[{'name': 'tavily_search', 'args': {'query': 'today'}, 'id': 'c2407249-04ce-4cd1-a495-079874acf8e5', 'type': 'tool_call'}]


In [61]:
query= " How to treat lung cancer? "
response= llm_with_tools.invoke(query)
print(response)
print(response.tool_calls)

content='' additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T11:37:15.894038Z', 'done': True, 'done_reason': 'stop', 'total_duration': 731704459, 'load_duration': 24597584, 'prompt_eval_count': 285, 'prompt_eval_duration': 184113042, 'eval_count': 20, 'eval_duration': 522609833, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-a3a8615b-31e5-47d3-bebb-d2b81de416cf-0' tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'lung cancer treatment'}, 'id': 'b4e70d23-cc3d-4eed-90b3-09d20b8f3b8c', 'type': 'tool_call'}] usage_metadata={'input_tokens': 285, 'output_tokens': 20, 'total_tokens': 305}
[{'name': 'pubmed_search', 'args': {'query': 'lung cancer treatment'}, 'id': 'b4e70d23-cc3d-4eed-90b3-09d20b8f3b8c', 'type': 'tool_call'}]


## generate final result

In [73]:
from langchain_core.messages import HumanMessage

In [74]:
query= " How to treat lung cancer? "
messages= [HumanMessage(query)]
tool_calls= llm_with_tools.invoke(messages).tool_calls

In [75]:
tool_calls

[{'name': 'pubmed_search',
  'args': {'query': 'lung cancer treatment'},
  'id': '90092d6b-a9aa-4174-b612-e269ec985342',
  'type': 'tool_call'}]

In [76]:
messages.append(tool_calls)

In [77]:
messages

[HumanMessage(content=' How to treat lung cancer? ', additional_kwargs={}, response_metadata={}),
 [{'name': 'pubmed_search',
   'args': {'query': 'lung cancer treatment'},
   'id': '90092d6b-a9aa-4174-b612-e269ec985342',
   'type': 'tool_call'}]]

In [78]:
query= " How to treat lung cancer? "
messages= [HumanMessage(query)]
ai_msg= llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [79]:
messages

[HumanMessage(content=' How to treat lung cancer? ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-01T11:45:45.810525Z', 'done': True, 'done_reason': 'stop', 'total_duration': 733581750, 'load_duration': 62084792, 'prompt_eval_count': 285, 'prompt_eval_duration': 93681792, 'eval_count': 22, 'eval_duration': 577495083, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f3fec5c0-9859-4131-a12a-3fe45cf7914d-0', tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'treatment of lung cancer'}, 'id': 'b7f2eaf9-0291-4e52-be37-2fb356868918', 'type': 'tool_call'}], usage_metadata={'input_tokens': 285, 'output_tokens': 22, 'total_tokens': 307})]

In [84]:
for tool_call in ai_msg.tool_calls:
    print(tool_call)
    name= tool_call['name'].lower()
    selected_tool= list_of_tools[name]
    tool_msg=selected_tool.invoke(tool_call)
    messages.append(tool_msg)

{'name': 'pubmed_search', 'args': {'query': 'treatment of lung cancer'}, 'id': 'b7f2eaf9-0291-4e52-be37-2fb356868918', 'type': 'tool_call'}


In [87]:
response= llm_with_tools.invoke(messages)
print(response.content)

**Lung Cancer Treatment: A Comprehensive Overview**

Lung cancer is a leading cause of cancer-related deaths worldwide. The treatment of lung cancer depends on the stage, type, and location of the tumor, as well as the overall health of the patient. Here's a summary of the current treatment options:

**Surgery**

* Lobectomy (removal of the lobe) or pneumonectomy (removal of the entire lung)
* Wedge resection (removal of a small portion of the tumor)
* Stereotactic body radiation therapy (SBRT) for early-stage cancer

**Chemotherapy**

* Non-small cell lung cancer (NSCLC): carboplatin, paclitaxel, etoposide, and gemcitabine
* Small cell lung cancer (SCLC): etoposide and platinum-based chemotherapy

**Targeted Therapy**

* NSCLC: erlotinib, gefitinib, and afatinib for EGFR-positive tumors
* SCLC: doxorubicin and topotecan for extensive disease

**Immunotherapy**

* NSCLC: pembrolizumab, nivolumab, and atezolizumab for PD-L1 positive tumors
* SCLC: pembrolizumab and nivolumab for extensi